# Notebook: Working with Our PostgreSQL Database:

## **Setup:**

In [14]:
# Import libraries we will use:
from dotenv import load_dotenv
import os
import pandas as pd
import psycopg2

In [15]:
# Get access info from .env file:
load_dotenv()

# PostgreSQL DB info:
PG_DB_HOST = os.getenv("PG_DB_HOST")
PG_DB_PORT = os.getenv("PG_DB_PORT")
PG_DB_NAME = os.getenv("PG_DB_NAME")
PG_DB_USER = os.getenv("PG_DB_USER")
PG_DB_PW = os.getenv("PG_DB_PW")
PG_DB_URI = os.getenv("PG_DB_URI")

In [16]:
# Open a connection to our PostgreSQL DB:
pg_conn = psycopg2.connect(
    host = PG_DB_HOST,
    port = PG_DB_PORT,
    database = PG_DB_NAME,
    user = PG_DB_USER,
    password = PG_DB_PW
)

# Instantiate a cursor using this connection:
pg_cursor = pg_conn.cursor()

## Functions: Get Info From Our PostgreSQL DB:

In [17]:
def get_feedback_for_user(user_id:int):
    """
    Using the user's user_id in our database (= 'owner_id' in the 'feedback' table),
    looks up and returns all feedback on all of that user's videos.
    """

    # Check to make sure input param video_id is the right type:
    if type(user_id) is not int:
        raise ValueError('Invalid user_id')

    # Get all feedback for the given video from our PostgreSQL DB:
    pg_cursor.execute(f"SELECT * FROM feedback WHERE owner_id={user_id};")
    column_names = [column_name[0] for column_name in pg_cursor.description]

    feedback_dataframe = pd.DataFrame(data=pg_cursor.fetchall(), columns=column_names)

    return feedback_dataframe

In [18]:
def get_feedback_for_video(video_id:int):
    """
    Using the video's video_id in our database, looks up and
    returns all feedback on that video.
    """

    # Check to make sure input param video_id is the right type:
    if type(video_id) is not int:
        raise ValueError('Invalid video_id')

    # Get all feedback for the given video from our PostgreSQL DB:
    pg_cursor.execute(f"SELECT * FROM feedback WHERE video_id={video_id};")
    column_names = [column_name[0] for column_name in pg_cursor.description]

    feedback_dataframe = pd.DataFrame(data=pg_cursor.fetchall(), columns=column_names)

    return feedback_dataframe

In [19]:
def get_video_info(video_s3_key:str):
    """
    Using the video's S3 key, looks up info for that video, user and prompt
    in our PostgreSQL database, and returns all info in a dictionary with
    the following top-level keys:
    {
        'video': {
            info for the video itself
        },
        'prompt': {
            info for the prompt the video was responding to
        },
        'user': {
            includes info for the user who posted the video
        }
    }
    """

    # Return error if the input video_s3_key param is not a string:
    if type(video_s3_key) is not str:
        raise ValueError('Invalid video_s3_key')

    video_info = {'video': {},
                  'prompt': {},
                  'user': {}}

    # Lookup info for this video and user in our DB:

    # From DB table: 'videos'
    lookup_table_videos = lookup_in_videos_table(video_s3_key)
    if lookup_table_videos != "No data for this key in this table.":
        # Video info: Add video info to the video_info dict we will return:
        video_info['video']['video_id'] = lookup_table_videos['id']
        video_info['video']['title'] = lookup_table_videos['title']
        s3_key = lookup_table_videos['video_url']
        video_info['video']['s3_key'] = s3_key
        video_info['video']['s3_filename'] = s3_key.split('/')[-1]
        video_info['video']['created_at'] = lookup_table_videos['created_at']
        video_info['video']['updated_at'] = lookup_table_videos['updated_at']

        video_info['user']['user_id'] = lookup_table_videos['owner_id']
        video_info['prompt']['prompt_id'] = lookup_table_videos['prompt_id']

        # From DB table: 'users'
        lookup_table_users = lookup_in_users_table(video_info['user']['user_id'])
        if lookup_table_users != "No data for this key in this table.":
            # User info: Add user info to the video_info dict we will return:
            first_name = lookup_table_users['first_name'].capitalize()
            last_name = lookup_table_users['last_name'].capitalize()
            video_info['user']['first_name'] = first_name
            video_info['user']['last_name'] = last_name
            video_info['user']['name'] = first_name + " " + last_name
            video_info['user']['username'] = lookup_table_users['username']

        # From DB table: 'prompts'
        lookup_table_prompts = lookup_in_prompts_table(video_info['prompt']['prompt_id'])
        if lookup_table_prompts != "No data for this key in this table.":
            # Prompt info: Add prompt/question info to the the video_info dict:
            video_info['prompt']['question'] = lookup_table_prompts['question']

    return video_info

In [20]:
def lookup_in_videos_table(video_s3_key):
    """
    Using an S3 key for a video (e.g., from a message in our SQS queue),
    get the info for this video from our DB table 'videos' and return it as a dict.
    """

    pg_cursor.execute(f"SELECT * FROM videos WHERE video_url='{video_s3_key}';")
    results = pg_cursor.fetchall()

    # Check to make sure results are not empty (i.e., no data in this table):
    if results == []:
        return "No data for this key in this table."

    # If there is data for this user_id in this table, return it:
    values = results[0]
    column_names = [column_name[0] for column_name in pg_cursor.description]
    results = dict(zip(column_names, values))

    return results

In [21]:
def lookup_in_users_table(user_id):
    """
    Using a user_id in our TeamReel production DB (= owner_id in the 'users' table),
    get the info for this user from our DB table 'users' and return it as a dict.
    """

    pg_cursor.execute(f"SELECT * FROM users WHERE id={user_id};")
    results = pg_cursor.fetchall()

    # Check to make sure results are not empty (i.e., no data in this table):
    if results == []:
        return "No data for this key in this table."

    # If there is data for this user_id in this table, return it:
    values = results[0]
    column_names = [column_name[0] for column_name in pg_cursor.description]
    results = dict(zip(column_names, values))

    return results

In [22]:
def lookup_in_prompts_table(prompt_id):
    """
    Using a user_id in our TeamReel production DB (= owner_id in the 'users' table),
    get the info for this user from our DB table 'users' and return it as a dict.
    """

    pg_cursor.execute(f"SELECT * FROM prompts WHERE id={prompt_id};")
    results = pg_cursor.fetchall()

    # Check to make sure results are not empty (i.e., no data in this table):
    if results == []:
        return "No data for this key in this table."

    # If there is data for this user_id in this table, return it:
    values = results[0]
    column_names = [column_name[0] for column_name in pg_cursor.description]
    results = dict(zip(column_names, values))

    return results

## Testing Our Functions:

In [23]:
# get_feedback_for_user(user_id:int):
get_feedback_for_user(user_id=185)

,id,post,video_id,owner_id,created_at,updated_at,viewed
0,134,"If you did nothing, nothing would have been done.",130,185,2020-04-29 19:51:30.622000+00:00,2020-04-29 19:51:30.622000+00:00,True
1,140,Great eye contact!,135,185,2020-05-05 21:35:28.962000+00:00,2020-05-05 21:35:28.962000+00:00,True


In [24]:
# get_feedback_for_video(video_id:int):
get_feedback_for_video(video_id=134)

,id,post,video_id,owner_id,created_at,updated_at,viewed
0,135,this is cool!,134,191,2020-05-05 16:23:30.411000+00:00,2020-05-05 16:23:30.411000+00:00,True
1,136,Great eye contact!,134,199,2020-05-05 16:23:37.899000+00:00,2020-05-05 16:23:37.899000+00:00,True
2,137,Feedback,134,192,2020-05-05 16:23:52.631000+00:00,2020-05-05 16:23:52.631000+00:00,True
3,138,feedback,134,200,2020-05-05 16:24:16.773000+00:00,2020-05-05 16:24:16.773000+00:00,True
4,139,feedback\n,134,193,2020-05-05 16:24:44.452000+00:00,2020-05-05 16:24:44.452000+00:00,True


In [25]:
# get_video_info(video_s3_key:str):
get_video_info(video_s3_key='videos/ALPACAVID-i7swK-Wzc.webm')

{'video': {'video_id': 134,
  'title': 'Kyla Oyamot',
  's3_key': 'videos/ALPACAVID-i7swK-Wzc.webm',
  's3_filename': 'ALPACAVID-i7swK-Wzc.webm',
  'created_at': datetime.datetime(2020, 5, 5, 16, 22, 56, 852000, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)),
  'updated_at': datetime.datetime(2020, 5, 5, 16, 22, 56, 852000, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None))},
 'prompt': {'prompt_id': 65,
  'question': 'What is your first impression of Labs'},
 'user': {'user_id': 185,
  'first_name': 'Kyla',
  'last_name': 'O',
  'name': 'Kyla O',
  'username': 'kylao'}}

In [26]:
# lookup_in_videos_table(video_s3_key):
lookup_in_videos_table(video_s3_key='videos/ALPACAVID-i7swK-Wzc.webm')

{'id': 134,
 'owner_id': 185,
 'title': 'Kyla Oyamot',
 'description': 'what i think about labs.',
 'created_at': datetime.datetime(2020, 5, 5, 16, 22, 56, 852000, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)),
 'updated_at': datetime.datetime(2020, 5, 5, 16, 22, 56, 852000, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)),
 'video_url': 'videos/ALPACAVID-i7swK-Wzc.webm',
 'prompt_id': 65,
 'thumbnail': None}

In [27]:
# lookup_in_users_table(user_id):
lookup_in_users_table(user_id=185)

{'id': 185,
 'email': 'kyla-oyamot@lambdastudents.com',
 'username': 'kylao',
 'password': '$2b$08$sLmhqbRgkN6QejsqaNfMxOXbOSYqiek3xRcLU7Dy6rcYgvqjraPu6',
 'first_name': 'kyla',
 'last_name': 'o',
 'avatar': 'avatars/superhero-2.png'}

In [28]:
# lookup_in_prompts_table(prompt_id):
lookup_in_prompts_table(prompt_id=65)

{'id': 65,
 'question': 'What is your first impression of Labs',
 'description': '5 minute video.',
 'team_id': 79,
 'created_at': datetime.datetime(2020, 5, 5, 16, 21, 5, 700000, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)),
 'updated_at': datetime.datetime(2020, 5, 5, 16, 21, 5, 700000, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None))}

## Get Info for 1 Video From Our DB (Multiple Tables):

In [29]:
# Which video:
video_s3_key = 'videos/ALPACAVID-i7swK-Wzc.webm'

In [30]:
video_info = get_video_info(video_s3_key)
video_info

{'video': {'video_id': 134,
  'title': 'Kyla Oyamot',
  's3_key': 'videos/ALPACAVID-i7swK-Wzc.webm',
  's3_filename': 'ALPACAVID-i7swK-Wzc.webm',
  'created_at': datetime.datetime(2020, 5, 5, 16, 22, 56, 852000, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)),
  'updated_at': datetime.datetime(2020, 5, 5, 16, 22, 56, 852000, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None))},
 'prompt': {'prompt_id': 65,
  'question': 'What is your first impression of Labs'},
 'user': {'user_id': 185,
  'first_name': 'Kyla',
  'last_name': 'O',
  'name': 'Kyla O',
  'username': 'kylao'}}

In [31]:
video_id = video_info['video']['video_id']
get_feedback_for_video(video_id=video_id)

,id,post,video_id,owner_id,created_at,updated_at,viewed
0,135,this is cool!,134,191,2020-05-05 16:23:30.411000+00:00,2020-05-05 16:23:30.411000+00:00,True
1,136,Great eye contact!,134,199,2020-05-05 16:23:37.899000+00:00,2020-05-05 16:23:37.899000+00:00,True
2,137,Feedback,134,192,2020-05-05 16:23:52.631000+00:00,2020-05-05 16:23:52.631000+00:00,True
3,138,feedback,134,200,2020-05-05 16:24:16.773000+00:00,2020-05-05 16:24:16.773000+00:00,True
4,139,feedback\n,134,193,2020-05-05 16:24:44.452000+00:00,2020-05-05 16:24:44.452000+00:00,True


In [32]:
user_id = video_info['user']['user_id']
get_feedback_for_user(user_id=user_id)

,id,post,video_id,owner_id,created_at,updated_at,viewed
0,134,"If you did nothing, nothing would have been done.",130,185,2020-04-29 19:51:30.622000+00:00,2020-04-29 19:51:30.622000+00:00,True
1,140,Great eye contact!,135,185,2020-05-05 21:35:28.962000+00:00,2020-05-05 21:35:28.962000+00:00,True


## Take a Look at All of the Tables in Our DB:

In [33]:
def get_db_table_as_dataframe(table_name):
    """
    Return the specified table in our database as a Pandas dataframe.
    """
    # Get info from the specified table: All rows (fetchall()):
    pg_cursor.execute(f"SELECT * FROM {table_name}")
    column_names = [column_name[0] for column_name in pg_cursor.description]
    
    table_as_dataframe = pd.DataFrame(data=pg_cursor.fetchall(), columns=column_names)
    
    return table_as_dataframe

In [34]:
# Get the names of all tables in our DB:
pg_cursor.execute(
    """
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'public';
    """
)

db_table_names = sorted([table_name[0] for table_name in pg_cursor.fetchall()])
db_table_names

['avatars',
 'feedback',
 'knex_migrations',
 'knex_migrations_lock',
 'organizations',
 'organizations_users',
 'prompts',
 'roles',
 'team_invite_link',
 'team_members',
 'teams',
 'users',
 'videos']

In [35]:
index = 0
table_name = db_table_names[index]

In [36]:
print(f"\nDATABASE TABLE: '{table_name.upper()}' \n")
get_db_table_as_dataframe(table_name)


DATABASE TABLE: 'AVATARS' 



,id,src
0,1,avatars/superhero-1.png
1,2,avatars/superhero-10.png
2,3,avatars/superhero-11.png
3,4,avatars/superhero-12.png
4,5,avatars/superhero-2.png
5,6,avatars/superhero-3.png
6,7,avatars/superhero-4.png
7,8,avatars/superhero-5.png
8,9,avatars/superhero-6.png
9,10,avatars/superhero-7.png


In [37]:
index +=1
table_name = db_table_names[index]

In [38]:
print(f"\nDATABASE TABLE: '{table_name.upper()}' \n")
get_db_table_as_dataframe(table_name)


DATABASE TABLE: 'FEEDBACK' 



,id,post,video_id,owner_id,created_at,updated_at,viewed
0,1,consequat morbi a ipsum integer a nibh in,28,13,2019-04-20 18:59:30+00:00,2019-09-14 17:15:17+00:00,True
1,2,pede ac diam cras pellentesque volutpat dui ma...,84,18,2019-11-24 12:28:43+00:00,2019-10-25 12:38:50+00:00,True
2,3,nec nisi vulputate nonummy maecenas tincidunt ...,15,17,2019-10-10 02:57:36+00:00,2019-04-15 11:44:36+00:00,True
3,4,euismod scelerisque quam turpis adipiscing lor...,28,35,2019-11-10 16:05:19+00:00,2019-02-12 17:24:39+00:00,True
4,5,in faucibus orci luctus et ultrices,57,96,2019-11-01 00:58:30+00:00,2019-12-08 07:18:59+00:00,True
...,...,...,...,...,...,...,...
138,135,this is cool!,134,191,2020-05-05 16:23:30.411000+00:00,2020-05-05 16:23:30.411000+00:00,True
139,136,Great eye contact!,134,199,2020-05-05 16:23:37.899000+00:00,2020-05-05 16:23:37.899000+00:00,True
140,137,Feedback,134,192,2020-05-05 16:23:52.631000+00:00,2020-05-05 16:23:52.631000+00:00,True
141,138,feedback,134,200,2020-05-05 16:24:16.773000+00:00,2020-05-05 16:24:16.773000+00:00,True


In [39]:
index +=1
table_name = db_table_names[index]

In [40]:
print(f"\nDATABASE TABLE: '{table_name.upper()}' \n")
get_db_table_as_dataframe(table_name)


DATABASE TABLE: 'KNEX_MIGRATIONS' 



,id,name,batch,migration_time
0,1,20200127183442_add-tables.js,1,2020-02-11 00:23:26.443000+00:00
1,2,20200129094834_videos-prompts.js,1,2020-02-11 00:23:26.474000+00:00
2,3,20200131122616_first-last-name.js,1,2020-02-11 00:23:26.484000+00:00
3,4,20200203151652_feedback.js,1,2020-02-11 00:23:26.498000+00:00
4,5,20200205121043_team-invite-link.js,1,2020-02-11 00:23:26.510000+00:00
5,6,20200206142237_avatars.js,1,2020-02-11 00:23:26.525000+00:00
6,7,20200214134707_video-thumbnails.js,2,2020-02-18 18:42:59.772000+00:00
7,8,20200408090610_add-column-to-feedback.js,3,2020-04-28 20:25:49.036000+00:00
8,9,20200409090100_change-fb-viewed-default.js,3,2020-04-28 20:25:49.061000+00:00
9,10,20200415125620_organization.js,3,2020-04-28 20:25:49.124000+00:00


In [41]:
index +=1
table_name = db_table_names[index]

In [42]:
print(f"\nDATABASE TABLE: '{table_name.upper()}' \n")
get_db_table_as_dataframe(table_name)


DATABASE TABLE: 'KNEX_MIGRATIONS_LOCK' 



,index,is_locked
0,1,0


In [43]:
index +=1
table_name = db_table_names[index]

In [44]:
print(f"\nDATABASE TABLE: '{table_name.upper()}' \n")
get_db_table_as_dataframe(table_name)


DATABASE TABLE: 'ORGANIZATIONS' 



,id,name
0,1,Organization ONE
1,2,IsaOrg1
2,3,Best Org
3,4,Great Org 2
4,5,blah
5,6,blah2
6,7,Labs22
7,8,Labs23
8,9,TestOrg
9,10,Carlo & Company Property Management


In [45]:
index +=1
table_name = db_table_names[index]

In [46]:
print(f"\nDATABASE TABLE: '{table_name.upper()}' \n")
get_db_table_as_dataframe(table_name)


DATABASE TABLE: 'ORGANIZATIONS_USERS' 



,organization_id,user_id,role_id
0,1,4,4
1,1,5,4
2,1,6,4
3,1,7,4
4,1,8,4
...,...,...,...
196,21,205,3
197,23,206,3
198,24,207,3
199,25,208,3


In [47]:
index +=1
table_name = db_table_names[index]

In [48]:
print(f"\nDATABASE TABLE: '{table_name.upper()}' \n")
get_db_table_as_dataframe(table_name)


DATABASE TABLE: 'PROMPTS' 



,id,question,description,team_id,created_at,updated_at
0,1,Tell me a little about yourself.,Maecenas pulvinar lobortis est. Phasellus sit ...,1,2019-03-07 01:33:02+00:00,2019-04-22 13:16:51+00:00
1,2,Why did you decide to get into _track_ and bec...,Suspendisse ornare consequat lectus.,1,2019-06-26 23:51:56+00:00,2019-08-12 09:21:58+00:00
2,3,Tell me about the most recent project you work...,Praesent blandit lacinia erat. Praesent blandi...,2,2019-10-02 12:56:37+00:00,2020-01-10 06:40:48+00:00
3,4,Desribe a time you were able to improve upon t...,Vivamus in felis eu sapien cursus vestibulum.,2,2019-12-02 17:38:27+00:00,2019-04-15 07:09:48+00:00
4,5,Tell me about the project you are most proud o...,Nam nulla.,3,2019-04-28 18:42:47+00:00,2019-05-02 23:23:41+00:00
...,...,...,...,...,...,...
69,70,How did you do,Tell me about it,92,2020-05-12 20:35:22.018000+00:00,2020-05-12 20:35:22.018000+00:00
70,71,question 1,basic question,96,2020-05-14 19:09:07.509000+00:00,2020-05-14 19:09:07.509000+00:00
71,72,question 2,basic question,96,2020-05-14 19:09:38.859000+00:00,2020-05-14 19:09:38.859000+00:00
72,73,What to do?,i don't konw,98,2020-05-18 05:56:56.095000+00:00,2020-05-18 05:56:56.095000+00:00


In [49]:
index +=1
table_name = db_table_names[index]

In [50]:
print(f"\nDATABASE TABLE: '{table_name.upper()}' \n")
get_db_table_as_dataframe(table_name)


DATABASE TABLE: 'ROLES' 



,id,name
0,1,Team Member
1,2,Team Lead
2,3,Organization Owner
3,4,Organization Member


In [51]:
index +=1
table_name = db_table_names[index]

In [52]:
print(f"\nDATABASE TABLE: '{table_name.upper()}' \n")
get_db_table_as_dataframe(table_name)


DATABASE TABLE: 'TEAM_INVITE_LINK' 



,id,team_id,link,isValid,created_at,expires_at,organization_id
0,49,71,team-PsiNuChi,True,2020-04-29 17:57:41.394000+00:00,2020-04-30 17:57:41.394000+00:00,5
1,50,74,Alpacavids-MuOmicronEta,True,2020-04-29 19:46:56.162000+00:00,2020-04-30 19:46:56.162000+00:00,7
2,51,24,Alpaca-BetaEtaEpsilon,True,2020-05-04 05:43:37.671000+00:00,2020-05-05 05:43:37.671000+00:00,1
3,52,79,Team-GammaUpsilonMu,True,2020-05-05 16:19:26.995000+00:00,2020-05-06 16:19:26.995000+00:00,12
4,53,80,General-SigmaTauEpsilon,True,2020-05-05 21:34:18.218000+00:00,2020-05-06 21:34:18.218000+00:00,13
...,...,...,...,...,...,...,...
56,57,90,General-ChiAlphaRho,True,2020-05-06 17:17:46.279000+00:00,2020-05-07 17:17:46.279000+00:00,20
57,58,86,New-ZetaThetaGamma,True,2020-05-06 17:36:09.336000+00:00,2020-05-07 17:36:09.336000+00:00,15
58,59,91,testing-OmicronOmicronDelta,True,2020-05-08 00:14:42.322000+00:00,2020-05-09 00:14:42.323000+00:00,12
59,60,99,General-OmegaOmicronMu,True,2020-05-19 21:56:05.780000+00:00,2020-05-20 21:56:05.780000+00:00,23


In [53]:
index +=1
table_name = db_table_names[index]

In [54]:
print(f"\nDATABASE TABLE: '{table_name.upper()}' \n")
get_db_table_as_dataframe(table_name)


DATABASE TABLE: 'TEAM_MEMBERS' 



,user_id,team_id,role_id
0,1,20,2
1,1,2,1
2,2,4,2
3,2,7,1
4,3,17,2
...,...,...,...
238,205,98,2
239,206,99,2
240,207,100,2
241,208,101,2


In [55]:
index +=1
table_name = db_table_names[index]

In [56]:
print(f"\nDATABASE TABLE: '{table_name.upper()}' \n")
get_db_table_as_dataframe(table_name)


DATABASE TABLE: 'TEAMS' 



,id,name,description,created_at,updated_at,avatar,organization_id,team_type
0,63,My personal team,My personal team,2020-04-28 20:42:47.312724+00:00,2020-04-28 20:42:47.312724+00:00,None,1,private
1,67,Superstars,Great team,2020-04-29 17:18:30.395687+00:00,2020-04-29 17:18:30.395687+00:00,None,3,private
2,70,General,This is a general team for all members,2020-04-29 17:57:02.223304+00:00,2020-04-29 17:57:02.223304+00:00,None,6,public
3,73,General,This is a general team for all members,2020-04-29 19:45:45.342592+00:00,2020-04-29 19:45:45.342592+00:00,None,8,public
4,74,Alpacavids,Alpacavids,2020-04-29 19:46:29.956266+00:00,2020-04-29 19:46:29.956266+00:00,None,7,private
...,...,...,...,...,...,...,...,...
97,72,General,This is a general team for all members,2020-04-29 19:45:22.245663+00:00,2020-04-29 19:45:22.245663+00:00,None,7,public
98,80,General,This is a general team for all members,2020-05-05 21:31:35.713589+00:00,2020-05-05 21:31:35.713589+00:00,None,13,public
99,84,General,This is a general team for all members,2020-05-05 21:41:59.294931+00:00,2020-05-05 21:41:59.294931+00:00,None,15,public
100,87,General,This is a general team for all members,2020-05-05 21:51:49.881234+00:00,2020-05-05 21:51:49.881234+00:00,None,17,public


In [57]:
index +=1
table_name = db_table_names[index]

In [58]:
print(f"\nDATABASE TABLE: '{table_name.upper()}' \n")
get_db_table_as_dataframe(table_name)


DATABASE TABLE: 'USERS' 



,id,email,username,password,first_name,last_name,avatar
0,2,oivashkin1@nsw.gov.au,tlefeaver1,$2b$08$IGM.eonWTK1wSIXyrBIo3eXaZTSXe3lu2haEHDO...,Leopold,Varndall,avatars/superhero-2.png
1,3,jmuldowney2@cargocollective.com,jdudney2,$2b$08$tXKxhKKZK5pkLyRQHY6EmOFlddr2GVWW/.anonP...,Brenna,Shwalbe,avatars/superhero-3.png
2,4,lgunner3@tamu.edu,tvolke3,$2b$08$T.xlpRKR11sqprkcyGo7xOcSmG///5Esy54aFDy...,Grace,Eyer,avatars/superhero-4.png
3,5,balderton4@printfriendly.com,cdeem4,$2b$08$fRxFtvmy3dzc1P5lLOkSH.H69QNmbjiGQDUXIsO...,Idaline,Gounet,avatars/superhero-5.png
4,6,aabazi5@nps.gov,nnosworthy5,$2b$08$26g1XtRPVEHjJlncwX3E6uuZvcaZG8s7D5l9IxS...,Sasha,Leys,avatars/superhero-6.png
...,...,...,...,...,...,...,...
186,208,test@testing.com,testaccount,$2b$08$.P2ZeC8eI4ahmETr5O7hG.EpCXWoe8RiND5EPOz...,testing,123,avatars/superhero-10.png
187,209,bethtest2@gmail.com,bethtest2,$2b$08$4tddwOp16me.6leXhzByB.djIkiJbi/VF6LZa22...,beth,wingate,avatars/superhero-4.png
188,211,teamreel2020@gmail.com,teamreel,$2b$08$OHCN1UNS4wlJ35zwouXMa.Zdns1rU0DocrTHP1b...,Chris,Giroux,avatars/superhero-6.png
189,212,wengellen@gmail.com,ellen,$2b$08$a3ypvHKy4BOdf.yIi9.xH.n0C4VsjMnYFsok/eX...,ellen,weng,avatars/superhero-10.png


In [59]:
index +=1
table_name = db_table_names[index]

In [60]:
print(f"\nDATABASE TABLE: '{table_name.upper()}' \n")
get_db_table_as_dataframe(table_name)


DATABASE TABLE: 'VIDEOS' 



,id,owner_id,title,description,created_at,updated_at,video_url,prompt_id,thumbnail
0,1,7,sed sagittis nam,parturient montes nascetur ridiculus mus vivam...,2019-11-06 11:36:53+00:00,2019-11-20 12:50:02+00:00,https://www.youtube.com/embed/6Gw5dK48MtI,11,http://i3.ytimg.com/vi/6Gw5dK48MtI/hqdefault.jpg
1,2,76,amet,non mattis pulvinar nulla pede ullamcorper aug...,2019-04-02 21:23:26+00:00,2019-07-11 08:49:03+00:00,https://www.youtube.com/embed/ssR-RguvjHo,3,http://i3.ytimg.com/vi/ssR-RguvjHo/hqdefault.jpg
2,3,4,vulputate,pellentesque volutpat dui maecenas tristique e...,2019-04-05 06:20:43+00:00,2019-12-20 03:24:05+00:00,https://www.youtube.com/embed/ssR-RguvjHo,24,http://i3.ytimg.com/vi/ssR-RguvjHo/hqdefault.jpg
3,4,90,tellus,est et tempus semper est quam pharetra magna a...,2019-07-28 14:17:51+00:00,2019-04-26 09:12:49+00:00,https://www.youtube.com/embed/13zN4-MVM9g,5,http://i3.ytimg.com/vi/13zN4-MVM9g/hqdefault.jpg
4,5,52,quam,amet diam in magna bibendum imperdiet nullam o...,2019-02-10 16:39:29+00:00,2019-05-01 20:41:57+00:00,https://www.youtube.com/embed/fc3c3OrpKSI,21,http://i3.ytimg.com/vi/fc3c3OrpKSI/hqdefault.jpg
...,...,...,...,...,...,...,...,...,...
132,133,101,New video,My awesome video,2020-05-04 06:05:26.035000+00:00,2020-05-04 06:05:26.035000+00:00,videos/ALPACAVID-EpV8rudeP.webm,64,None
133,134,185,Kyla Oyamot,what i think about labs.,2020-05-05 16:22:56.852000+00:00,2020-05-05 16:22:56.852000+00:00,videos/ALPACAVID-i7swK-Wzc.webm,65,None
134,135,193,Naomi,Some quick thoughts.,2020-05-05 21:29:35.421000+00:00,2020-05-05 21:29:35.421000+00:00,videos/ALPACAVID-AKJ9GADkT.webm,66,None
135,136,196,ABC,It's easy as...,2020-05-05 21:54:45.351000+00:00,2020-05-05 21:54:45.351000+00:00,videos/ALPACAVID-gJmn4O_ud.webm,68,None


## **Close the Cursor and Connection:**

In [61]:
# Close the cursor:
pg_cursor.close()

In [62]:
# Close the connection:
pg_conn.close()